In [22]:
!pip install pretty_midi music21 gradio tensorflow


In [23]:
!apt-get install -y fluidsynth
!pip install pydub


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [24]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = "/content/midi_dataset"  # Change this to your actual folder path


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import os
import numpy as np
import pretty_midi
import music21
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import gradio as gr
import subprocess
from pydub import AudioSegment


In [26]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = "/content/midi_dataset"  # Change to your folder

def extract_multi_instrument_notes(folder_path):
    all_data = []
    for file in os.listdir(folder_path):
        if file.endswith(".mid") or file.endswith(".midi"):
            try:
                midi = pretty_midi.PrettyMIDI(os.path.join(folder_path, file))
                for instrument in midi.instruments:
                    inst_name = pretty_midi.program_to_instrument_name(instrument.program).replace(" ", "_")
                    for note in instrument.notes:
                        if 21 <= note.pitch <= 108:
                            all_data.append(f"{inst_name}_{note.pitch}")
            except Exception as e:
                print(f"Skipped {file}: {e}")
    return all_data

notes = extract_multi_instrument_notes(folder_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [27]:
sequence_length = 100
unique_notes = sorted(set(notes))
note_to_int = {note: i for i, note in enumerate(unique_notes)}
int_to_note = {i: note for note, i in note_to_int.items()}

def prepare_sequences(notes, sequence_length):
    sequences = []
    targets = []
    for i in range(len(notes) - sequence_length):
        seq_in = notes[i:i + sequence_length]
        seq_out = notes[i + sequence_length]
        input_seq = [note_to_int[n] for n in seq_in]
        sequences.append(input_seq)
        targets.append(note_to_int[seq_out])
    return np.array(sequences), tf.keras.utils.to_categorical(targets, num_classes=len(note_to_int))

X, y = prepare_sequences(notes, sequence_length)
X = np.reshape(X, (X.shape[0], sequence_length, 1)) / float(len(note_to_int))


In [28]:
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(note_to_int), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=50, batch_size=64)  # You can reduce epochs to speed up training

# Save model
model.save('multi_instrument_lstm_model.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - loss: 4.8744
Epoch 2/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 3.4425
Epoch 3/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 3.0182
Epoch 4/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.8731
Epoch 5/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.7392
Epoch 6/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - loss: 2.6644
Epoch 7/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - loss: 2.5559
Epoch 8/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.4973
Epoch 9/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.4399
Epoch 10/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.3627
Epoch 11/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.3085
Epoch 12/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.2759
Epoch 13/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.2174
Epoch 14/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 2.1925
Epoch 15/50
499/499 ━━━━━━━━━

In [29]:
import random
from itertools import cycle

In [30]:
from collections import Counter

def generate_music(model, start_sequence, length=200, temperature=0.8):
    generated = list(start_sequence)
    instr_history = []

    for i in range(length):
        input_seq = [note_to_int[n] for n in generated[-sequence_length:]]
        input_seq = np.reshape(input_seq, (1, sequence_length, 1)) / float(len(note_to_int))
        preds = model.predict(input_seq, verbose=0)[0]

        # Apply temperature
        temp = temperature + (i / length) * 0.3  # slowly warm it up
        preds = np.log(preds + 1e-8) / temp
        preds = np.exp(preds) / np.sum(np.exp(preds))

        # Penalize last note repetition
        last_note = note_to_int.get(generated[-1], None)
        if last_note is not None:
            preds[last_note] *= 0.7

        # Penalize overused instruments
        recent_instrs = [n.split("_")[0] for n in generated[-30:]]
        instr_counts = Counter(recent_instrs)

        for idx, note in int_to_note.items():
            instr = note.split("_")[0]
            penalty = 0.8 ** instr_counts[instr]  # exponential decay
            preds[idx] *= penalty

        preds = preds / np.sum(preds)
        next_index = np.random.choice(range(len(preds)), p=preds)
        next_note = int_to_note[next_index]
        generated.append(next_note)
        instr_history.append(next_note.split("_")[0])

    return generated




In [31]:
def notes_to_midi(predicted_notes, selected_instruments, output_path='generated_song.mid'):
    selected_instruments = [instr for instr in selected_instruments if instr != "None"]
    midi_stream = music21.stream.Stream()

    if not selected_instruments:
        selected_instruments = ["Piano"]  # fallback

    instrument_cycle = cycle(selected_instruments)
    instrument_parts = {name: music21.stream.Part() for name in selected_instruments}
    for name in selected_instruments:
        instr_class = getattr(music21.instrument, name, music21.instrument.Piano)
        instrument_parts[name].insert(0, instr_class())

    for note_token in predicted_notes:
        try:
            _, pitch = note_token.split("_")
            pitch = int(pitch)
            note = music21.note.Note(pitch)
            note.quarterLength = 0.5
            current_instrument = next(instrument_cycle)
            instrument_parts[current_instrument].append(note)
        except:
            continue

    for part in instrument_parts.values():
        midi_stream.append(part)

    midi_stream.write('midi', fp=output_path)

    # Convert MIDI to WAV using FluidSynth
    wav_output_path = output_path.replace(".mid", ".wav")
    soundfont_path = "/usr/share/sounds/sf2/FluidR3_GM.sf2"
    subprocess.run(["fluidsynth", "-ni", soundfont_path, output_path, "-F", wav_output_path, "-r", "44100"])

    return output_path, wav_output_path



In [32]:
model = tf.keras.models.load_model('multi_instrument_lstm_model.h5')  # Load trained model

# Updated generation function
def generate_and_play_music(temperature, num_notes, instr1, instr2, instr3):
    seed_index = random.randint(0, len(notes) - sequence_length - 1)
    seed = notes[seed_index:seed_index + sequence_length]
    generated_notes = generate_music(model, seed, length=num_notes, temperature=temperature)
    midi_path, wav_path = notes_to_midi(generated_notes, [instr1, instr2, instr3])
    return wav_path, midi_path




# Add "None" to instrument choices
instrument_list = ["None"] + sorted([
    "Piano", "Violin", "Flute", "Guitar", "Cello", "Clarinet", "Trumpet", "Saxophone",
    "Trombone", "Oboe", "Harp", "Bassoon", "Tuba", "Accordion", "Xylophone"
])

# UI Components
temperature_slider = gr.Slider(0.2, 2.0, value=0.8, step=0.01, label="Temperature")
note_slider = gr.Slider(100, 2000, value=200, step=10, label="Number of Notes")

instrument1 = gr.Dropdown(choices=instrument_list, value="Piano", label="Instrument 1")
instrument2 = gr.Dropdown(choices=instrument_list, value="Violin", label="Instrument 2")
instrument3 = gr.Dropdown(choices=instrument_list, value="Flute", label="Instrument 3")

# Gradio Interface
interface = gr.Interface(
    fn=generate_and_play_music,
    inputs=[
        temperature_slider,
        note_slider,
        instrument1,
        instrument2,
        instrument3
    ],
    outputs=[
        gr.Audio(label="🔊 Live Playback (WAV)", type="filepath"),
        gr.File(label="🎼 Download MIDI")
    ],
    live=False,
    title="🎹 AI Music Generator (Multi-Instrument Harmony)",
    description="Choose up to 3 instruments (or fewer using 'None'), adjust temperature & notes, and generate music! 🎶"
)

interface.launch(share=True, show_api=False)




Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://45b78a898455b793c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
